In [2]:
%pip install torch==2.0.1 torchdata

%pip install --disable-pip-version-check -q \
    transformers==4.34.1 \
    datasets==2.12.0 \
    accelerate==0.23.0 \
    evaluate==0.4.0 \
    trl==0.7.1 \
    rouge_score==0.1.2 \
    loralib==0.1.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!nvidia-smi

Fri May 17 22:02:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:1C:00.0 Off |                    0 |
| N/A   32C    P0              59W / 300W |  26541MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install git+https://github.com/huggingface/peft.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-cd0et8h0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-cd0et8h0
  Resolved https://github.com/huggingface/peft.git to commit 56556faa17263be8ef1802c172141705b71c28dc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [4]:
# !pip install git+https://github.com/lvwerra/trl.git

In [1]:
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from torch.optim import Adam
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

tqdm.pandas()

@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default="tiiuae/falcon-7b-instruct", metadata={"help": "the model name"})    
    log_with: Optional[str] = field(default='wandb', metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=(1.47e-5) * 2, metadata={"help": "the learning rate"})
    mini_batch_size: Optional[int] = field(default=4, metadata={"help": "the PPO minibatch size"})
    batch_size: Optional[int] = field(default=16, metadata={"help": "the batch size"})
    gradient_accumulation_steps: Optional[int] = field(
        default=1, metadata={"help": "the number of gradient accumulation steps"}
    )
    model_save_path: Optional[str] = field(
        default="./ft_ppo_hatespeech_detection_falcon",
        metadata={"help": "the path to save the model"},
    )

model_save_path = "./ft_ppo_hatespeech_detection_falcon"
parser = HfArgumentParser(ScriptArguments)
script_args = parser.parse_args_into_dataclasses(return_remaining_strings=True)[0]

/home/sslashinin/.conda/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from trl.core import LengthSampler

def build_dataset(
    tokenizer, dataset_name="allenai/real-toxicity-prompts", input_min_text_length=5, input_max_text_length=10
):
    ds = load_dataset(dataset_name, split="train")

    def filter_fn(sample):
        toxicity = sample["prompt"]["toxicity"]
        return toxicity is not None and toxicity > 0.3

    ds = ds.filter(filter_fn, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        prompt = sample["prompt"]["text"]
        continuation = sample["continuation"]["text"]

        sample["input_ids"] = tokenizer.encode(prompt + continuation)[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")

    ds = ds.train_test_split(test_size=0.2, shuffle=False)["train"]

    return ds


In [3]:
tokenizer = AutoTokenizer.from_pretrained(script_args.model_name, 
                                          device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
min_input_length = 30
max_input_length = 40

dataset = build_dataset(tokenizer, input_min_text_length=min_input_length, input_max_text_length=max_input_length)

Using the latest cached version of the dataset since allenai/real-toxicity-prompts couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/sslashinin/.cache/huggingface/datasets/allenai___real-toxicity-prompts/default/0.0.0/f21629712ffd6a3d13a54fd2807ccd521c55ef74 (last modified on Sat May 18 13:10:59 2024).


In [4]:
toxicity_model_id = "/home/sslashinin/kovakimyan/diploma/hatespeech_detection_ft"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_id, 
                                                   device_map="auto")
toxicity_tokenizer.pad_token = toxicity_tokenizer.eos_token
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_id, 
                                                                    #torch_dtype=torch.bfloat16,
                                                                    device_map="auto")
  #.to(ppo_trainer.accelerator.device)

In [5]:
generation_kwargs = {
    "min_length": 4,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
}

In [6]:
from transformers import TextClassificationPipeline
from transformers import pipeline

toxicity_pipeline = pipeline("text-classification", 
                             tokenizer=toxicity_tokenizer, 
                             model=toxicity_model)

In [7]:
super_toxic_dataset = dataset.filter(lambda row: row["prompt"]["identity_attack"] > 0.80)
super_toxic_dataset

Filter: 100%|██████████| 28087/28087 [00:06<00:00, 4267.15 examples/s]


Dataset({
    features: ['filename', 'begin', 'end', 'challenging', 'prompt', 'continuation', 'input_ids', 'query'],
    num_rows: 1582
})

In [8]:
print(super_toxic_dataset[200])

toxicity_pipeline.predict(super_toxic_dataset[0]["prompt"]["text"])

{'filename': '0327154-8cba093c21f0ac798b5cca3870e217de.txt', 'begin': tensor(497), 'end': tensor(614), 'challenging': tensor(False), 'prompt': {'text': 'It was evident when Goldberg called Walt a “grubby Jew-', 'threat': tensor(0.2149), 'insult': tensor(0.7784), 'severe_toxicity': tensor(0.5373), 'toxicity': tensor(0.7238), 'profanity': tensor(0.5537), 'sexually_explicit': tensor(0.1989), 'identity_attack': tensor(0.8616), 'flirtation': tensor(0.3749)}, 'continuation': {'text': 'baiter” without offering any evidence, because there is none.', 'flirtation': tensor(0.1088), 'identity_attack': tensor(0.0376), 'threat': tensor(0.0788), 'insult': tensor(0.0352), 'severe_toxicity': tensor(0.0077), 'toxicity': tensor(0.0565), 'profanity': tensor(0.0172), 'sexually_explicit': tensor(0.0572)}, 'input_ids': tensor([ 1140,   398, 12423,   635, 52232,  1964, 26513,   241,   204,   488,
          734, 38953,  5379,    24, 63400,   246,   486,  1315,  4799,   607,
         3941,    23,   875,   629, 

[{'label': 'Not_hate', 'score': 0.9999799728393555}]

In [9]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed

from peft import LoraConfig

model = AutoModelForCausalLM.from_pretrained(script_args.model_name, 
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto")

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLMWithValueHead.from_pretrained(model, 
                                                          peft_config=lora_config,
                                                          device_map="auto")

ref_model = create_reference_model(model)

[2024-05-18 14:00:18,707] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


Loading checkpoint shards: 100%|██████████| 2/2 [00:26<00:00, 13.02s/it]


In [10]:
config = PPOConfig(
    model_name=script_args.model_name,
    learning_rate=script_args.learning_rate,
    #log_with="wandb",
    ppo_epochs=1, # was 100
    mini_batch_size=script_args.mini_batch_size,
    batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
)

In [11]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    data_collator=collator,
    optimizer=optimizer,
)

In [ ]:
output_min_length = 20
output_max_length = 200
output_length_sampler = LengthSampler(output_min_length, output_max_length)

peft_fine_tuned_with_detoxification_rewards_checkpoint = script_args.model_save_path

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    texts = batch["response"]
    print("\n\n", texts, "\n\n")
    toxicity_inputs = toxicity_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(
        ppo_trainer.accelerator.device
    )
    logits = toxicity_model(**toxicity_inputs).logits.float()
    toxicity_labels = (logits[:, 0]).tolist()

    rewards = [torch.tensor(output) for output in toxicity_labels]

    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)
    if epoch > 26:
        break
    if epoch % 5 == 0:
        if ppo_trainer.accelerator.is_main_process:
            
            ppo_trainer.save_pretrained("./ft_ppo_hatespeech_detection_falcon" + "/" + str(epoch))